In [1]:
PIPELINE_CONTEXT = {"_runtime_context_name": "PIPELINE_CONTEXT", "params": {"n_clusters": "auto", "n_clusters_min": 6, "n_clusters_max": 30, "n_clusters_criterion": "silhouette", "pca_components": "auto", "pca_variance": 0.9, "pca_max_components": 50, "random_state": 42}, "artifacts": {"dir": "/vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436", "processed_data": "/vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/processed.parquet", "model_file": "/vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/kmeans_model.joblib", "scaler_file": "/vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/scaler.joblib", "pca_file": "/vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/pca.joblib", "cluster_summary": "/vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/cluster_summary.json", "elbow_plot": "/vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/elbow_plot.png", "silhouette_plot": "/vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/silhouette_plot.png", "db_plot": "/vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/db_plot.png", "ch_plot": "/vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/ch_plot.png", "selection": "/vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/selection.json"}, "cwd": "/vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/pipeline", "data_dir": "/vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/data/output_by_college_clean"}

**Pipeline Context**

In [2]:
from pathlib import Path
import json

try:
    ctx = PIPELINE_CONTEXT  # injected by orchestrator
except NameError:
    # Fallbacks for manual runs
    ctx = {
        "params": {},
        "artifacts": {
            "dir": "artifacts",
            "processed_data": "artifacts/processed.parquet",
            "model_file": "artifacts/kmeans_model.joblib",
            "scaler_file": "artifacts/scaler.joblib",
            "pca_file": "artifacts/pca.joblib",
            "cluster_summary": "artifacts/cluster_summary.json",
            "elbow_plot": "artifacts/elbow_plot.png",
            "silhouette_plot": "artifacts/silhouette_plot.png",
            "selection": "artifacts/selection.json",
        },
        # set to your data root when running manually
        "data_dir": "data/output_by_college_clean",
    }

P = ctx.get("params", {})
PATHS = ctx.get("artifacts", {})
DATA_DIR = ctx.get("data_dir", "data")  

Path(PATHS["dir"]).mkdir(parents=True, exist_ok=True)

def save_json(obj, path):
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w") as f:
        json.dump(obj, f, indent=2, default=float)

def load_json(path, default=None):
    p = Path(path)
    if p.exists():
        return json.loads(p.read_text())
    return {} if default is None else default


In [3]:
# Imports and Configuration  
import glob  
import os  
import pandas as pd  
import sqlite3 
import numpy as np
from pathlib import Path

In [4]:
# Paths and patterns (anchored to DATA_DIR)
base = Path(DATA_DIR)  # injected by orchestrator/config

# exactly 2 levels: <college>/<season>/file.csv
indiv_pattern = str(base / "*" / "*" / "individual_stats_overall.csv")
team_pattern  = str(base / "*" / "*" / "team_stats.csv")

print("DATA_DIR:", base.resolve())
print("Patterns:\n ", indiv_pattern, "\n ", team_pattern)


DATA_DIR: /vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/data/output_by_college_clean
Patterns:
  /vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/data/output_by_college_clean/*/*/individual_stats_overall.csv 
  /vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/data/output_by_college_clean/*/*/team_stats.csv


In [5]:
directory = Path("data/output_by_college" )
csv_count = len(list(directory.rglob('*.csv')))
print(f"Number of CSV files: {csv_count}")


Number of CSV files: 0


In [6]:
# College rename map  
def get_college_map():  
    return {  
        'gojaspers_com':           "manhattan",  
        'mountathletics_com':      "mount st marys",  
        'purpleeagles_com':        "niagara",  
        'sacredheartpioneers_com': "sacred heart",  
        'gobobcats_com':           "quinnipiac",  
        'merrimackathletics_com':  "merrimack",  
        'goredfoxes_com':          "marist",  
        'fairfieldstags_com':      "fairfield",  
        'ionagaels_com':           "iona",  
        'sienasaints_com':         "siena",  
        'gogriffs_com':            "canisius",  
        'saintpeterspeacocks_com': "saint peters",  
        'gobroncs_com':            "rider",  
    }  

college_map = get_college_map()

In [7]:
# Slugify helper  
def slugify(col: str) -> str:  
    """Convert column names to lowercase underscores and replace % and spaces."""  
    s = col.strip().lower()  
    s = s.replace('%', '_pct').replace('/', '_per_')  
    for ch in [' ', '-', ':', '(', ')']:  
        s = s.replace(ch, '_')  
    while '__' in s:  
        s = s.replace('__', '_')  
    return s


In [8]:
# Build master_individual_stats.csv
indiv_paths = glob.glob(indiv_pattern, recursive=False)  # <-- no recursive needed now
if not indiv_paths:
    # optional fallback if some teams are nested deeper than expected
    indiv_paths = glob.glob(str(base / "**" / "individual_stats_overall.csv"), recursive=True)

if not indiv_paths:
    raise FileNotFoundError(
        f"No individual CSVs found under {base.resolve()}\n"
        f"Tried: {indiv_pattern}"
    )

indiv_dfs = []
for path in indiv_paths:
    parts = Path(path).parts  # robust split
    if len(parts) < 3:
        print(f"Skipping {path} (unexpected layout)")
        continue

    college = parts[-3]
    season  = parts[-2]

    df = pd.read_csv(path)
    df.columns = [slugify(c) for c in df.columns]
    df['college'] = college
    df['season']  = season
    indiv_dfs.append(df)

if not indiv_dfs:
    raise ValueError("No valid individual CSVs parsed.")

df_indiv = pd.concat(indiv_dfs, ignore_index=True)
df_indiv['college'] = df_indiv['college'].replace(college_map)

# Save to artifacts (run folder)
out_ind_csv = Path(PATHS["dir"]) / "master_individual_stats.csv"
df_indiv.to_csv(out_ind_csv, index=False)
print('Saved master_individual_stats.csv to', out_ind_csv, df_indiv.shape)
print(df_indiv.head())


Saved master_individual_stats.csv to /vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/master_individual_stats.csv (2065, 28)
   player_number          player  gp    gs  minutes_tot  minutes_avg  fg_fgm  \
0              4   Kayla Hoohuli  30  30.0         1001         33.4     136   
1              5   Tiahana Mills  30  29.0          972         32.4     108   
2             24  Crystal Porter  30  30.0          754         25.1     127   
3             31     Emily Weber  30  29.0          722         24.1      84   
4             20   Jasmine Mungo  27  22.0          504         18.7      50   

   fg_fga  fg_fg_pct  3pt  ...  rebounds_def  rebounds_tot  rebounds_avg  pf  \
0     305      0.446   66  ...            77            99           3.3  48   
1     246      0.439   21  ...            94           114           3.8  59   
2     280      0.454    0  ...           116           177           5.9  82   
3     218      0.385   44  

In [9]:
# Build master_team_stats.csv
team_paths = glob.glob(team_pattern, recursive=False)
if not team_paths:
    team_paths = glob.glob(str(base / "**" / "team_stats.csv"), recursive=True)

if not team_paths:
    raise FileNotFoundError(
        f"No team CSVs found under {base.resolve()}\n"
        f"Tried: {team_pattern}"
    )


records = []
for path in team_paths:
    parts   = Path(path).parts
    if len(parts) < 3:
        print(f"Skipping {path} (unexpected layout)")
        continue

    college = parts[-3]
    season  = parts[-2]

    df    = pd.read_csv(path)
    total = df.iloc[0]

    rec = {'college': college, 'season': season}
    for raw_col, val in total.items():
        if pd.isna(raw_col) or str(raw_col).strip() == '':
            continue
        key = slugify(raw_col)
        rec[key] = val
    records.append(rec)

df_team = pd.DataFrame(records)
df_team['college'] = df_team['college'].replace(college_map)

cols = ['college', 'season'] + [c for c in df_team.columns if c not in ('college','season')]
df_team = df_team[cols]

# Save to artifacts (run folder)
out_team_csv = Path(PATHS["dir"]) / "master_team_stats.csv"
df_team.to_csv(out_team_csv, index=False)
print('Saved master_team_stats.csv to', out_team_csv, df_team.shape)
print(df_team.head())


Saved master_team_stats.csv to /vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/master_team_stats.csv (152, 28)
    college   season  player_number player  gp  gs  minutes_tot  minutes_avg  \
0  canisius  2014-15            NaN  Total  30 NaN         6050        201.7   
1  canisius  2017-18            NaN  Total  30 NaN         6050        201.7   
2  canisius  2024-25            NaN  Total  31 NaN         6207        200.2   
3  canisius  2018-19            NaN  Total  30 NaN         6100        203.3   
4  canisius  2020-21            NaN  Total   5 NaN         1000        200.0   

   fg_fgm  fg_fga  ...  scoring_avg  rebounds_off  rebounds_def  rebounds_tot  \
0     670    1639  ...        61.50           321           663           984   
1     598    1636  ...        55.53           292           769          1061   
2     607    1567  ...        55.06           316           771          1087   
3     644    1655  ...        57.60 

In [10]:
# # Load CSVs into SQLite  
# DB_PATH = 'wbb_stats.db'  
# conn    = sqlite3.connect(DB_PATH)  

# pd.read_csv('master_individual_stats.csv').to_sql('individual_stats', conn, if_exists='replace', index=False)  

# pd.read_csv('master_team_stats.csv').to_sql('team_stats', conn, if_exists='replace', index=False)  

# with conn:  
#     conn.execute('CREATE INDEX IF NOT EXISTS idx_indiv ON individual_stats(college, season)')  
#     conn.execute('CREATE INDEX IF NOT EXISTS idx_team  ON team_stats     (college, season)')  

# tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)  
# print('Tables in DB:', tables['name'].tolist())  
# conn.close()


In [11]:
# Load CSVs into SQLite
from pathlib import Path
import sqlite3, pandas as pd

DB_PATH = Path(PATHS["dir"]) / "wbb_stats.db"
conn = sqlite3.connect(DB_PATH)

ind_csv  = Path(PATHS["dir"]) / "master_individual_stats.csv"
team_csv = Path(PATHS["dir"]) / "master_team_stats.csv"

pd.read_csv(ind_csv).to_sql('individual_stats', conn, if_exists='replace', index=False)
pd.read_csv(team_csv).to_sql('team_stats',        conn, if_exists='replace', index=False)

with conn:
    conn.execute('CREATE INDEX IF NOT EXISTS idx_indiv ON individual_stats(college, season)')
    conn.execute('CREATE INDEX IF NOT EXISTS idx_team  ON team_stats     (college, season)')

tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print('Tables in DB:', tables['name'].tolist(), "at", DB_PATH)
conn.close()


Tables in DB: ['individual_stats', 'team_stats'] at /vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/wbb_stats.db


In [12]:
# Load merged inputs from artifacts (CSV first; DB fallback)
ind_csv  = Path(PATHS["dir"]) / "master_individual_stats.csv"
team_csv = Path(PATHS["dir"]) / "master_team_stats.csv"

if ind_csv.exists() and team_csv.exists():
    df_i = pd.read_csv(ind_csv)
    df_t = pd.read_csv(team_csv)
else:
    DB_PATH = Path(PATHS["dir"]) / "wbb_stats.db"
    if not DB_PATH.exists():
        raise FileNotFoundError(
            f"Neither CSVs nor DB found in artifacts dir: {PATHS['dir']}\n"
            f"Expected {ind_csv.name}, {team_csv.name} or {DB_PATH.name}"
        )
    conn = sqlite3.connect(DB_PATH)
    df_i = pd.read_sql('SELECT * FROM individual_stats', conn)
    df_t = pd.read_sql('SELECT * FROM team_stats',        conn)
    conn.close()

# Merge on college & season; individual/team suffixes
df = df_i.merge(df_t, on=['college','season'], how='left', suffixes=('_ind','_team'))


In [13]:
# Load from SQLite
# DB_PATH = Path(PATHS["dir"]) / "wbb_stats.db"
# if not DB_PATH.exists():
#     raise FileNotFoundError(f"DB not found: {DB_PATH}")

# conn = sqlite3.connect(DB_PATH)
# df_i = pd.read_sql('SELECT * FROM individual_stats', conn)
# df_t = pd.read_sql('SELECT * FROM team_stats',        conn)
# conn.close()



# # Merge on college & season; individual/team suffixes
# df = df_i.merge(df_t, on=['college','season'], how='left', suffixes=('_ind','_team'))


# Helper to pick the correct column name (unchanged)
def pick(base_names, suf):
    for base in base_names:
        name = f"{base}{suf}"
        if name in df.columns:
            return name
    raise KeyError(f"No column found among {[b+suf for b in base_names]}")

# Identify key columns (with suffix) — your existing picks
pn_col   = pick(['player_number'], '_ind')
p_col    = pick(['player'],        '_ind')
gp_col   = pick(['gp'],            '_ind')
gs_col   = pick(['gs'],            '_ind')
m_ind    = pick(['minutes_tot','minutes'], '_ind')
m_team   = pick(['minutes_tot','minutes'], '_team')

pts_ind  = pick(['scoring_pts','pts'],         '_ind')
reb_tot  = pick(['rebounds_tot','reb'],         '_ind')
ast_ind  = pick(['ast'],                       '_ind')
stl_ind  = pick(['stl'],                       '_ind')
blk_ind  = pick(['blk'],                       '_ind')
to_ind   = pick(['to','tov'],                  '_ind')

fgm_ind  = pick(['fg_fgm','fgm'],              '_ind')
fga_ind  = pick(['fg_fga','fga'],              '_ind')
three_ind= pick(['3pt'],                       '_ind')
fta_ind  = pick(['ft_fta','fta'],              '_ind')

fga_team = pick(['fg_fga','fga'],              '_team')
fta_team = pick(['ft_fta','fta'],              '_team')
to_team  = pick(['to','tov'],                  '_team')

# Compute per-40 (unchanged)
df['pts_per40'] = np.where(df[m_ind]>0, df[pts_ind]/df[m_ind]*40, 0)
df['reb_per40'] = np.where(df[m_ind]>0, df[reb_tot]/df[m_ind]*40, 0)
df['ast_per40'] = np.where(df[m_ind]>0, df[ast_ind]/df[m_ind]*40, 0)
df['stl_per40'] = np.where(df[m_ind]>0, df[stl_ind]/df[m_ind]*40, 0)
df['blk_per40'] = np.where(df[m_ind]>0, df[blk_ind]/df[m_ind]*40, 0)
df['to_per40'] = np.where(df[m_ind]>0, df[to_ind]/df[m_ind]*40, 0)

# Effective FG%
df['eFG_pct'] = np.where(df[fga_ind]>0,
    (df[fgm_ind] + 0.5*df[three_ind]) / df[fga_ind], 0)

# True Shooting%
df['TS_pct'] = np.where((df[fga_ind] + 0.44*df[fta_ind])>0,
    df[pts_ind] / (2*(df[fga_ind] + 0.44*df[fta_ind])), 0)

# Usage Rate%
u_num = df[fga_ind] + 0.44*df[fta_ind] + df[to_ind]
u_den = df[fga_team] + 0.44*df[fta_team] + df[to_team]
df['USG_pct'] = np.where((df[m_ind]>0)&(u_den>0),
    100 * (u_num * df[m_team])/(df[m_ind]*u_den), 0)

# Rebound Rates
df['ORB_pct'] = np.where(
    (df['rebounds_off_ind']+df['rebounds_def_team'])>0,
    100*df['rebounds_off_ind']/(df['rebounds_off_ind']+df['rebounds_def_team']),0)
df['DRB_pct'] = np.where(
    (df['rebounds_def_ind']+df['rebounds_off_team'])>0,
    100*df['rebounds_def_ind']/(df['rebounds_def_ind']+df['rebounds_off_team']),0)

# Assist-to-Turnover & Assist Rate
df['AST_per_TO'] = np.where(df[to_ind]>0, df[ast_ind]/df[to_ind], np.nan)
df['AST_pct']    = np.where((df[ast_ind]+df[to_ind])>0,
    100*df[ast_ind]/(df[ast_ind]+df[to_ind]), 0)

# Enrichement Block
# 2-pt / 3-pt breakdown
df['fga_2pt'] = df[fga_ind] - df[three_ind]
df['fgm_2pt'] = df[fgm_ind] - df[three_ind]
df['2pt_pct'] = np.where(df['fga_2pt']>0,
    df['fgm_2pt'] / df['fga_2pt'],
    np.nan)

# Free-throw rate
df['FTr'] = np.where(df[fga_ind]>0,
    df[fta_ind] / df[fga_ind],
    0)

# Points per Field-Goal Attempt
df['PPP'] = np.where(df[fga_ind]>0,
    df[pts_ind] / df[fga_ind],
    np.nan)

# Estimate possessions
df['possessions'] = (
    df[fga_ind]
  + 0.44 * df[fta_ind]
  + df[to_ind]
)
df['possessions'].replace(0, np.nan, inplace=True)

# Per-100 possession rates
for stat in ['scoring_pts_ind','ast_ind','rebounds_tot_ind','stl_ind','blk_ind','to_ind']:
    name = stat.replace('_ind','') + '_per100'
    df[name] = df[stat] / df['possessions'] * 100




/tmp/ipykernel_808255/1620892997.py:108: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['possessions'].replace(0, np.nan, inplace=True)


In [14]:
# Per‐Game Averages
# Compute basic per‐game numbers to capture raw volume vs. rate trade‐offs
df['mins_per_game'] = df[m_ind]    / df[gp_col]
df['pts_per_game']  = df[pts_ind]  / df[gp_col]
df['ast_per_game']  = df[ast_ind]  / df[gp_col]
df['reb_per_game']  = df[reb_tot]  / df[gp_col]
df['stl_per_game']  = df[stl_ind]  / df[gp_col]
df['blk_per_game']  = df[blk_ind]  / df[gp_col]
df['to_per_game']   = df[to_ind]   / df[gp_col]


In [15]:
# Share Metrics
# What fraction of team production each player accounts for
for stat in ['scoring_pts','ast','rebounds_tot','stl','blk','to']:
    ind  = f"{stat}_ind"
    team = f"{stat}_team"
    df[f"{stat}_share"] = np.where(
        df[team] > 0,
        df[ind] / df[team],
        0
    )


In [16]:
team_columns = [col for col in df.columns if "_team" in col]
print("Columns with '_team' in the name:", team_columns)


Columns with '_team' in the name: ['player_number_team', 'player_team', 'gp_team', 'gs_team', 'minutes_tot_team', 'minutes_avg_team', 'fg_fgm_team', 'fg_fga_team', 'fg_fg_pct_team', '3pt_team', '3pt_3pta_team', '3pt_3pt_pct_team', 'ft_ftm_team', 'ft_fta_team', 'ft_ft_pct_team', 'scoring_pts_team', 'scoring_avg_team', 'rebounds_off_team', 'rebounds_def_team', 'rebounds_tot_team', 'rebounds_avg_team', 'pf_team', 'ast_team', 'to_team', 'stl_team', 'blk_team']


In [17]:
# Efficiency Differential
# Compute True Shooting Differential: how a player’s TS% compares to their team’s TS%

# Identify the correct team‐level columns using your pick() helper
pts_team = pick(['scoring_pts','pts'], '_team')
fga_team = pick(['fg_fga','fga'],        '_team')
fta_team = pick(['ft_fta','fta'],        '_team')

# Compute the team’s True Shooting %
df['team_TS_pct'] = np.where(
    (df[fga_team] + 0.44 * df[fta_team]) > 0,
    df[pts_team] / (2 * (df[fga_team] + 0.44 * df[fta_team])),
    0
)

# Subtract to get the differential
df['TS_diff'] = df['TS_pct'] - df['team_TS_pct']


In [18]:
# Assist & Turnover Ratios
# Assists per made field‐goal; turnover rate per opportunity
df['ast_per_fgm'] = np.where(
    df[fgm_ind] > 0,
    df[ast_ind] / df[fgm_ind],
    0
)
df['tov_rate'] = np.where(
    (df[fga_ind] + 0.44*df[fta_ind] + df[to_ind]) > 0,
    df[to_ind] / (df[fga_ind] + 0.44*df[fta_ind] + df[to_ind]),
    0
)


In [19]:
# Hollinger’s Game Score
# Single‐number box‐score impact metric
# (requires you have ft_ftm_ind or adjust if named differently)
df['game_score'] = (
     df[pts_ind]
   + 0.4 * df[fgm_ind]
   - 0.7 * df[fga_ind]
   - 0.4 * (df[fta_ind] - df['ft_ftm_ind'])
   + 0.7 * df[reb_tot]
   + 0.7 * df[ast_ind]
   + 0.7 * df[stl_ind]
   + 0.7 * df[blk_ind]
   - 0.4 * df['pf_ind']     # if you have personal fouls
   - df[to_ind]
)
df['game_score_per40'] = np.where(
    df[m_ind] > 0,
    df['game_score'] / df[m_ind] * 40,
    0
)


In [20]:
# Usage & Pace Shares
# Minutes share and possessions share
df['min_share'] = np.where(
    df[m_team] > 0,
    df[m_ind] / (df[m_team] / df[gp_col]),
    0
)
# If you computed team possessions earlier, use that column name:
# df['poss_share'] = df['possessions'] / df['team_possessions']

In [21]:
# Three‐point metrics 

# first, re-run your pick() helper to find the right column names
three_made = '3pt_ind'
three_att  = '3pt_3pta_ind'
three_pct = '3pt_3pt_pct_ind'

# compute 3-point attempt rate (3PA/FGA)
df['threeA_rate'] = np.where(
    df[fga_ind]>0,
    df[three_att] / df[fga_ind],
    0
)


# per‐40 stats
df['three_per40']   = np.where(df[m_ind] > 0,
                               df[three_made]/df[m_ind]*40, 0)
df['threeA_per40']  = np.where(df[m_ind] > 0,
                               df[three_att]/df[m_ind]*40, 0)

# per‐100 possessions
df['three_per100']  = np.where(df['possessions'] > 0,
                               df[three_made]/df['possessions']*100, 0)

df['threeA_per100'] = np.where(df['possessions'] > 0,
                               df[three_att]/df['possessions']*100, 0)



In [22]:
# Advanced Defensive Metrics
# df['def_rating_proxy']=df['stl_per100']+ df['blk_per100'] + df['rebounds_def_ind'] / df['possessions'] * 100

# Defensive Rebound Conversion Rate (DRCR)
df['DRCR'] = df['AST_per_TO'] * df['DRB_pct']

# Steal-to-Turnover Ratio (STL/TO)
df['STL_TO_ratio'] = np.where(
    df[to_ind] > 0,
    df[stl_ind] / df[to_ind],
    0
)

# Defensive Stops Per 100 Possessions
df['def_stops_per100'] = np.where(
    df['possessions'] > 0,
    df['stl_per100'] + df['blk_per100'] + (df['DRB_pct'] * df['possessions'] / 100),
    0
)

# Defensive Playmaking Rate (DPMR)
df['DPMR'] = np.where(
    df['minutes_tot_ind'] > 0,
    (df['stl_per40'] + df['blk_per40'] + 0.5 * df['stl_per40']) / df['minutes_tot_ind'],
    0
)

In [23]:
# Advanced Basketball Metrics 

# True Usage Rate (includes assists in offensive load)
df['TUSG_pct'] = (
    df['threeA_per40'] + df['pts_per40'] + 0.44 * df.get('FTr', 0) +
    df['to_per40'] + df['ast_per40'] * 0.5
) / (df['possessions'] + 1e-5)

# Scoring Gravity (shooting volume * efficiency * usage)
df['Gravity'] = df['threeA_rate'] * df['TS_pct'] * df['USG_pct']

# Points Per Touch (proxy for scoring efficiency per possession involvement)
df['PPT'] = df['PPP'] / (df['USG_pct'] + 1e-5)

# Floor Spacing Index (3PT volume and efficiency)
df['Spacing'] = df['threeA_per100'] * df['3pt_3pt_pct_ind']

# Assist-to-Usage Ratio (passing efficiency relative to ball dominance)
df['Assist_to_Usage'] = df['AST_pct'] / (df['USG_pct'] + 1e-5)

# Assist Points Created (estimated points generated via assists)
df['APC'] = df['ast_per_game'] * df['team_TS_pct'] * 2.0

# Playmaking Efficiency (creation quality without mistakes)
df['PEF'] = df['AST_per_TO'] * df['AST_pct']

# Offensive Efficiency (points per 100 possessions)
df['OEFF'] = df['scoring_pts_per100'] / (df['possessions'] + 1e-5)

# Turnover Rate (TOs per possession used)
df['TOV_pct'] = df['to_ind'] / (
    df['threeA_per100'] + 0.44 * df.get('FTr', 0) + df['to_ind'] + 1e-5
)

# Shooting Efficiency Margin (TS% relative to team average)
df['SEM'] = df['TS_pct'] - df['team_TS_pct']

# Possession Efficiency Index (summary of offensive value per possession)
df['PEI'] = (
    df['scoring_pts_per100'] + df['ast_per100'] * 2 - df['to_per100']
) / (df['possessions'] + 1e-5)

# Box Creation (estimated number of shots created via playmaking and gravity)
df['BoxCreation'] = df['ast_per100'] + 5 * (df['USG_pct'] * df['TS_pct'])

# Offensive Load Index (how many possessions a player "uses")
df['OLI'] = (
    df['threeA_per100'] + 0.44 * df.get('FTr', 0) +
    df['to_per100'] + df['ast_per100']
)

# Impact Per Minute (summary impact scaled by time played)
df['IPM'] = (
    df['scoring_pts_per100'] + df['ast_per100'] +
    df['stl_per100'] + df['blk_per100'] - df['to_per100']
) / (df['minutes_tot_ind'] + 1e-5)

# # Replace any remaining NaN/None with 0 (if a column was missing values)
# df.fillna(0, inplace=True)

# # Show updated dataframe
# df.head()


In [24]:
# Export Cell

# Base columns you already had before enrichment
out_cols_before = [
    'college','season',
    pn_col, p_col,
    gp_col, gs_col,
    m_ind, pts_ind, reb_tot, ast_ind, stl_ind, blk_ind, to_ind, 
    'pts_per40','reb_per40','ast_per40','stl_per40','blk_per40','to_per40',
    'eFG_pct','TS_pct','USG_pct','ORB_pct','DRB_pct','AST_pct','AST_per_TO',
    three_pct, 'three_per40', 'threeA_per40', 'three_per100','threeA_rate', 'DRCR',
    'STL_TO_ratio', 'def_stops_per100', 'DPMR'
# ]    
  , 'TUSG_pct', 'Gravity','PPT',
    'Spacing', 'Assist_to_Usage', 'APC','PEF', 'OEFF', 'TOV_pct', 'SEM',
    'PEI', 'BoxCreation', 'OLI', 'IPM','threeA_per100'
]

# Columns from enrichment block 
enrich1 = [
    '2pt_pct','FTr','PPP','possessions',
    'scoring_pts_per100','ast_per100','rebounds_tot_per100',
    'stl_per100','blk_per100','to_per100'
]
enrich2 = [
    # per-game stats
    'mins_per_game','pts_per_game','ast_per_game',
    'reb_per_game','stl_per_game','blk_per_game','to_per_game',
    # share metrics
    'scoring_pts_share','ast_share','rebounds_tot_share',
    'stl_share','blk_share','to_share',
    # efficiency differential
    'team_TS_pct','TS_diff',
    # assist/turnover ratios
    'ast_per_fgm','tov_rate',
    # game score
    'game_score','game_score_per40',
    # usage & pace shares
    'min_share'  # add 'poss_share' here if you calculated team possessions
]

# Combine them all
out_cols = out_cols_before + enrich1 + enrich2

# Drop any raw columns you no longer need
# drop_cols = ['fg_fga_ind','fg_fgm_ind','3pt_3pta_ind','3pt_ind','ft_fta_ind','ft_ftm_ind']
# df.drop(columns=[c for c in drop_cols if c in df], inplace=True)

# Export to CSV
df.to_csv('player_features.csv', columns=out_cols, index=False)

#________________________________________________________________________-
# Also copy into artifacts
out_pf = Path(PATHS["dir"]) / "player_features.csv"
df[out_cols].to_csv(out_pf, index=False)
print("Also wrote to artifacts:", out_pf)
#________________________________________________________________________-

# Sanity check
print(f"Written player_features.csv with shape {df[out_cols].shape}")
print("Columns exported:")
for col in out_cols:
    print("  -", col)


Also wrote to artifacts: /vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/player_features.csv
Written player_features.csv with shape (2065, 80)
Columns exported:
  - college
  - season
  - player_number_ind
  - player_ind
  - gp_ind
  - gs_ind
  - minutes_tot_ind
  - scoring_pts_ind
  - rebounds_tot_ind
  - ast_ind
  - stl_ind
  - blk_ind
  - to_ind
  - pts_per40
  - reb_per40
  - ast_per40
  - stl_per40
  - blk_per40
  - to_per40
  - eFG_pct
  - TS_pct
  - USG_pct
  - ORB_pct
  - DRB_pct
  - AST_pct
  - AST_per_TO
  - 3pt_3pt_pct_ind
  - three_per40
  - threeA_per40
  - three_per100
  - threeA_rate
  - DRCR
  - STL_TO_ratio
  - def_stops_per100
  - DPMR
  - TUSG_pct
  - Gravity
  - PPT
  - Spacing
  - Assist_to_Usage
  - APC
  - PEF
  - OEFF
  - TOV_pct
  - SEM
  - PEI
  - BoxCreation
  - OLI
  - IPM
  - threeA_per100
  - 2pt_pct
  - FTr
  - PPP
  - possessions
  - scoring_pts_per100
  - ast_per100
  - rebounds_tot_per100
  - stl_per100
 

In [25]:
df = pd.read_csv('player_features.csv')
print(df.shape)
df.head()

# Numeric overview
display(df.describe().T.assign(missing=lambda d: df.isna().sum()))

# Count of seasons & colleges
print("Seasons:", df['season'].nunique())
print("Colleges:", df['college'].nunique())

(2065, 80)


,count,mean,std,min,25%,50%,75%,max,missing
player_number_ind,2065.0,17.040678,12.395348,0.0,5.000000,15.000000,24.000000,55.000000,0
gp_ind,2065.0,20.311864,10.980011,1.0,11.000000,25.000000,30.000000,36.000000,0
gs_ind,2065.0,10.182082,11.746382,0.0,0.000000,3.000000,22.000000,35.000000,0
minutes_tot_ind,2065.0,410.265860,345.279436,0.0,71.000000,344.000000,709.000000,1225.000000,0
scoring_pts_ind,2065.0,123.766102,131.299955,0.0,14.000000,77.000000,202.000000,745.000000,0
...,...,...,...,...,...,...,...,...,...
ast_per_fgm,2065.0,0.635030,0.765202,0.0,0.222222,0.438462,0.793103,11.666667,0
tov_rate,2065.0,0.224778,0.145686,0.0,0.151741,0.206398,0.270479,1.000000,0
game_score,2065.0,89.354722,104.993069,-16.8,6.300000,48.000000,144.900000,629.500000,0
game_score_per40,2065.0,6.553574,7.023931,-80.0,3.256410,6.705596,10.001753,68.000000,0


Seasons: 21
Colleges: 13


In [26]:
# List columns with missing values
missing_columns = df.columns[df.isna().any()].tolist()
print("Columns with missing values:", missing_columns)


Columns with missing values: ['AST_per_TO', 'DRCR', 'TUSG_pct', 'PPT', 'PEF', 'OEFF', 'PEI', 'BoxCreation', 'OLI', 'IPM', '2pt_pct', 'PPP', 'possessions', 'scoring_pts_per100', 'ast_per100', 'rebounds_tot_per100', 'stl_per100', 'blk_per100', 'to_per100']


In [27]:
# Load your dataset

for col in missing_columns:
    print(f"Column: {col}")
    missing = df[df[col].isna()]
    missing_players = missing[['player_ind', 'season']]
    print(missing_players)
    print("\n")

Column: AST_per_TO
               player_ind   season
163            Lauren Saa  2019-20
170          Julia Duggan  2013-14
174   Shereen Lightbourne  2013-14
200           Lexi Stover  2016-17
225   Janine Loutzenhiser  2014-15
...                   ...      ...
1960       Jayla Leftwich  2012-13
1989    Chantell Gonzalez  2021-22
2025       Haley Blasetti  2018-19
2047      Rylee Carpenter  2022-23
2051        Simone Walker  2022-23

[144 rows x 2 columns]


Column: DRCR
               player_ind   season
163            Lauren Saa  2019-20
170          Julia Duggan  2013-14
174   Shereen Lightbourne  2013-14
200           Lexi Stover  2016-17
225   Janine Loutzenhiser  2014-15
...                   ...      ...
1960       Jayla Leftwich  2012-13
1989    Chantell Gonzalez  2021-22
2025       Haley Blasetti  2018-19
2047      Rylee Carpenter  2022-23
2051        Simone Walker  2022-23

[144 rows x 2 columns]


Column: TUSG_pct
                  player_ind   season
239           Sam Ric

In [28]:
for col in missing_columns:
    # Convert any infinities to NaN
    df[col] = df[col].replace([np.inf, -np.inf], np.nan)

    # Fill NaNs with 0 and assign back
    df[col] = df[col].fillna(0)

    # Check for any remaining missing values
    missing_counts = df.isna().sum()
    print("Columns with missing values:\n", missing_counts[missing_counts > 0])

# Confirm no missing values remain
display(df.describe().T.assign(missing=lambda d: df.isna().sum()))

Columns with missing values:
 DRCR                   144
TUSG_pct                35
PPT                     70
PEF                    144
OEFF                    35
PEI                     35
BoxCreation             35
OLI                     35
IPM                     35
2pt_pct                 72
PPP                     70
possessions             35
scoring_pts_per100      35
ast_per100              35
rebounds_tot_per100     35
stl_per100              35
blk_per100              35
to_per100               35
dtype: int64
Columns with missing values:
 TUSG_pct                35
PPT                     70
PEF                    144
OEFF                    35
PEI                     35
BoxCreation             35
OLI                     35
IPM                     35
2pt_pct                 72
PPP                     70
possessions             35
scoring_pts_per100      35
ast_per100              35
rebounds_tot_per100     35
stl_per100              35
blk_per100              35
to_per100

,count,mean,std,min,25%,50%,75%,max,missing
player_number_ind,2065.0,17.040678,12.395348,0.0,5.000000,15.000000,24.000000,55.000000,0
gp_ind,2065.0,20.311864,10.980011,1.0,11.000000,25.000000,30.000000,36.000000,0
gs_ind,2065.0,10.182082,11.746382,0.0,0.000000,3.000000,22.000000,35.000000,0
minutes_tot_ind,2065.0,410.265860,345.279436,0.0,71.000000,344.000000,709.000000,1225.000000,0
scoring_pts_ind,2065.0,123.766102,131.299955,0.0,14.000000,77.000000,202.000000,745.000000,0
...,...,...,...,...,...,...,...,...,...
ast_per_fgm,2065.0,0.635030,0.765202,0.0,0.222222,0.438462,0.793103,11.666667,0
tov_rate,2065.0,0.224778,0.145686,0.0,0.151741,0.206398,0.270479,1.000000,0
game_score,2065.0,89.354722,104.993069,-16.8,6.300000,48.000000,144.900000,629.500000,0
game_score_per40,2065.0,6.553574,7.023931,-80.0,3.256410,6.705596,10.001753,68.000000,0


In [29]:
# export cleaned dataframe
df.to_csv('player_features_cleaned.csv', index=False)
print("player_features_cleaned.csv saved, shape:", df.shape)

#________________________________________________________
# Also write to artifacts and parquet for the pipeline:
out_pf_clean = Path(PATHS["dir"]) / "player_features_cleaned.csv"
df.to_csv(out_pf_clean, index=False)
print("Also wrote to artifacts:", out_pf_clean)

# Canonical handoff for later notebooks/app:
df.to_parquet(PATHS["processed_data"])
print("Wrote canonical processed parquet:", PATHS["processed_data"])

player_features_cleaned.csv saved, shape: (2065, 80)


Also wrote to artifacts: /vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/player_features_cleaned.csv
Wrote canonical processed parquet: /vol/bitbucket/sn1624/data-driven-framework-for-athlete-profiling/artifacts/run_20250825_144436/processed.parquet


In [30]:
print(df.columns)

Index(['college', 'season', 'player_number_ind', 'player_ind', 'gp_ind',
       'gs_ind', 'minutes_tot_ind', 'scoring_pts_ind', 'rebounds_tot_ind',
       'ast_ind', 'stl_ind', 'blk_ind', 'to_ind', 'pts_per40', 'reb_per40',
       'ast_per40', 'stl_per40', 'blk_per40', 'to_per40', 'eFG_pct', 'TS_pct',
       'USG_pct', 'ORB_pct', 'DRB_pct', 'AST_pct', 'AST_per_TO',
       '3pt_3pt_pct_ind', 'three_per40', 'threeA_per40', 'three_per100',
       'threeA_rate', 'DRCR', 'STL_TO_ratio', 'def_stops_per100', 'DPMR',
       'TUSG_pct', 'Gravity', 'PPT', 'Spacing', 'Assist_to_Usage', 'APC',
       'PEF', 'OEFF', 'TOV_pct', 'SEM', 'PEI', 'BoxCreation', 'OLI', 'IPM',
       'threeA_per100', '2pt_pct', 'FTr', 'PPP', 'possessions',
       'scoring_pts_per100', 'ast_per100', 'rebounds_tot_per100', 'stl_per100',
       'blk_per100', 'to_per100', 'mins_per_game', 'pts_per_game',
       'ast_per_game', 'reb_per_game', 'stl_per_game', 'blk_per_game',
       'to_per_game', 'scoring_pts_share', 'ast_share